# Reproducibility study - Counterfactual Generative Networks
NOTE: Rerun the notebook see our actual results. Rendered results just show that the code runs. 

## Setup
We first need to install the packages 

In [34]:
import sys
py = sys.executable

In [78]:
!{py} -m pip install -r requirements.txt

     |████████████████████████████████| 489.6 MB 9.7 MB/s eta 0:00:011
     |████████████████████████████████| 5.7 MB 15.1 MB/s eta 0:00:01
     |████████████████████████████████| 365 kB 13.6 MB/s eta 0:00:01
     |████████████████████████████████| 4.1 MB 22.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 15.5 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 2.1 MB/s  eta 0:00:01
     |████████████████████████████████| 14.5 MB 21.1 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 6.4 MB/s  eta 0:00:01
     |████████████████████████████████| 126 kB 7.2 MB/s eta 0:00:01
     |████████████████████████████████| 463 kB 19.4 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 21.4 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 5.7 MB/s  eta 0:00:01
     |████████████████████████████████| 2.1 MB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 4.5 MB 67.5 MB/s eta 0:00:01
     |█████████████████████████████

In [1]:
import gdown

gdown.download('https://drive.google.com/u/0/uc?export=download&confirm=rHtT&id=1NSv4RCSHjcHois3dXjYw_PaLIoVlLgXu', 'colored_mnist.tar.gz')
gdown.download("https://drive.google.com/u/0/uc?id=1VkKexkWh5SeB8fgxAZxLKgmmvDXhVYUy&export=downloadl", "u2net.pth")
gdown.download("https://drive.google.com/u/0/uc?id=12yVFHPUjKmUFGnO2D4xVlTSpF8CUj136&export=download", "cgn.pth")
gdown.download("")

Downloading...
From: https://drive.google.com/u/0/uc?export=download&confirm=rHtT&id=1NSv4RCSHjcHois3dXjYw_PaLIoVlLgXu
To: /home/mund/FACT_CGN/colored_mnist.tar.gz
100%|██████████| 38.8M/38.8M [00:03<00:00, 10.9MB/s]
Downloading...
From: https://drive.google.com/u/0/uc?id=1VkKexkWh5SeB8fgxAZxLKgmmvDXhVYUy&export=downloadl
To: /home/mund/FACT_CGN/u2net.pth
100%|██████████| 176M/176M [00:17<00:00, 9.96MB/s] 
Downloading...
From: https://drive.google.com/u/0/uc?id=12yVFHPUjKmUFGnO2D4xVlTSpF8CUj136&export=download
To: /home/mund/FACT_CGN/cgn.pth
100%|██████████| 1.11G/1.11G [01:46<00:00, 10.4MB/s]


'cgn.pth'

In [86]:
%%sh
#!/usr/bin/env bash
# Move Colored MNIST
tar -xzvf colored_mnist.tar.gz 
mv colored_mnist mnists/data
rm colored_mnist.tar.gz

# Download BG challenge dataset
wget https://github.com/MadryLab/backgrounds_challenge/releases/download/data/backgrounds_challenge_data.tar.gz
tar -xzvf backgrounds_challenge_data.tar.gz
mkdir imagenet/data/in9
mv bg_challenge/* imagenet/data/in9
rmdir bg_challenge
rm backgrounds_challenge_data.tar.gz

# Download the Cue Conflict dataset
git clone https://github.com/rgeirhos/texture-vs-shape/
mkdir imagenet/data/cue_conflict
mv texture-vs-shape/stimuli/style-transfer-preprocessed-512/* imagenet/data/cue_conflict
rm -rf texture-vs-shape


In [4]:
%%sh
#!/usr/bin/env bash

mv u2net.pth imagenet/weights

mv cgn.pth imagenet/weights

wget "https://s3.amazonaws.com/models.huggingface.co/biggan/biggan-deep-256-pytorch_model.bin"
mv biggan-deep-256-pytorch_model.bin imagenet/weights/biggan256.pth

--2022-02-03 22:48:47--  https://s3.amazonaws.com/models.huggingface.co/biggan/biggan-deep-256-pytorch_model.bin
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.194.232
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.194.232|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 234411737 (224M) [application/octet-stream]
Saving to: ‘biggan-deep-256-pytorch_model.bin’

     0K .......... .......... .......... .......... ..........  0%  494K 7m44s
    50K .......... .......... .......... .......... ..........  0%  468K 7m56s
   100K .......... .......... .......... .......... ..........  0%  489K 7m53s
   150K .......... .......... .......... .......... ..........  0%  475K 7m55s
   200K .......... .......... .......... .......... ..........  0% 64.1M 6m21s
   250K .......... .......... .......... .......... ..........  0% 7.35M 5m22s
   300K .......... .......... .......... .......... ..........  0%  612K 5m30s
   350K .......... .......... .........

## Experiments

## MNIST

Before doing any MNIST operations we need to generate the non-counterfactual datasets.

In [ ]:
!{py} -u mnists/generate_data.py --dataset colored_MNIST
!{py} -u mnists/generate_data.py --dataset double_colored_MNIST
!{py} -u mnists/generate_data.py --dataset wildlife_MNIST

### Table 2

Note: generating table 2 can take upto 5 hours

In [4]:
!{py} experiments/table2.py

initialize network with orthogonal
Generating the counterfactual colored_MNIST of size 100 with cf_ratio 1
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.96it/s]
x shape torch.Size([100, 3, 32, 32]), y shape torch.Size([100])
Making model
Making dataloaders
EPOCH 1
Calling train function
Train Epoch: 1 [0/60100 (0%)]	Loss: 2.311327
Train Epoch: 1 [6400/60100 (11%)]	Loss: 0.596531
Train Epoch: 1 [12800/60100 (21%)]	Loss: 0.581550
Train Epoch: 1 [19200/60100 (32%)]	Loss: 0.399889
Train Epoch: 1 [25600/60100 (43%)]	Loss: 0.216951
Train Epoch: 1 [32000/60100 (53%)]	Loss: 0.362560
Train Epoch: 1 [38400/60100 (64%)]	Loss: 0.202984
Train Epoch: 1 [44800/60100 (74%)]	Loss: 0.196865
Train Epoch: 1 [51200/60100 (85%)]	Loss: 0.069814
Train Epoch: 1 [57600/60100 (96%)]	Loss: 0.071021

Train set: Average loss: 0.0012, Accuracy: 52556/60100 (87.4%)

Calling test function

Test set: Average loss: 6.9567, Accuracy: 3394/10000 (33.940%)

initialize network with orthogonal
Gene

In [6]:
!cat table2_data_2.json

{"original + CGN": {"colored_MNIST": [33.94, 87.44758735440932], "double_colored_MNIST": [10.23, 96.562874251497], "wildlife_MNIST": [10.47, 89.84231536926147]}, "CGN": {"colored_MNIST": [9.74, 11.0], "double_colored_MNIST": [9.67, 11.0], "wildlife_MNIST": [10.09, 8.0]}, "original + GAN": {"colored_MNIST": [28.02, 86.99833610648919], "double_colored_MNIST": [9.96, 97.27544910179641], "wildlife_MNIST": [10.31, 87.55489021956087]}, "GAN": {"colored_MNIST": [9.65, 10.0], "double_colored_MNIST": [9.52, 16.0], "wildlife_MNIST": [9.61, 9.0]}, "original": {"colored_MNIST": [34.98, 87.35], "double_colored_MNIST": [9.89, 96.998], "wildlife_MNIST": [10.26, 90.06]}}

### Heatmaps

NOTE: this experiments depends on the counterfactual dataset generated in the previous experiment.

In [15]:
!{py} mnists/generate_10_colored.py

In [17]:
!{py} mnists/train_classifier.py --dataset double_colored_MNIST --grad_cam
!{py} mnists/train_classifier.py --dataset double_colored_MNIST_counterfactual --grad_cam
!{py} mnists/train_classifier.py --dataset double_colored_MNIST_counterfactual --grad_cam  --original

Namespace(batch_size=64, dataset='double_colored_MNIST', epochs=1, gamma=0.7, grad_cam=True, guide_target=False, log_interval=100, lr=1.0, original=False, use_pretrained=False)
Making model
Making dataloaders
EPOCH 1
Calling train function
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.308064
Train Epoch: 1 [6400/50000 (13%)]	Loss: 0.000934
Train Epoch: 1 [12800/50000 (26%)]	Loss: 0.000030
Train Epoch: 1 [19200/50000 (38%)]	Loss: 0.000006
Train Epoch: 1 [25600/50000 (51%)]	Loss: 0.000036
Train Epoch: 1 [32000/50000 (64%)]	Loss: 0.000017
Train Epoch: 1 [38400/50000 (77%)]	Loss: 0.000010
Train Epoch: 1 [44800/50000 (90%)]	Loss: 0.000008

Train set: Average loss: 0.0000, Accuracy: 48675/50000 (97.3%)

Calling test function

Test set: Average loss: 33.8769, Accuracy: 10/100 (10.000%)

10.0 97.35

Test set: Average loss: 33.8769, Accuracy: 10/100 (10.000%)

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1.,

A sample heatmap (double colored MNIST):

In [25]:
!{py} mnists/plot_grad_cam.py

![heatmap](mnists/data/grad_cam/double_colored_MNIST_counterfactual_False_False/heatmap.png)

Note that ordering is OS specific 

## Imagenet

(Optional) If you want to train your own CGN remove the # from the command below. Note that training takes a lot of time (12 hours+)!

In [32]:
# !{py} imagenet/train_cgn.py --fill_textloss --model_name cgn_true_rand_fill_IN-mini --batch_acc 500 --episodes 200 --batch_sz 5 --log_losses --save_iter 1500

LAMBDA:
  BG: 2000
  BINARY: 300
  L1: 100
  MASK: 500
  PERC: [4, 4, 4, 4]
  TEXT: [0, 5, 5, 0]
LOG:
  LOSSES: True
  SAMPLED_FIXED_NOISE: False
  SAVE_ITER: 1
  SAVE_SINGLES: True
LR:
  BG: 1e-05
  SHAPE: 8e-06
  TEXTURE: 3e-05
MODEL:
  RES: 256
  TRUNCATION: 1.0
MODEL_NAME: cgn
TRAIN:
  BATCH_ACC: 2
  BATCH_SZ: 5
  EPISODES: 2
WEIGHTS_PATH: 
  0%|                                                     | 0/4 [00:00<?, ?it/s]

If you want to use your self trained CGN, change the path below to the location of the self trained CGN. Otherwhise you don't have to do anything :)

In [55]:
weight_path =  "imagenet/weights/cgn.pth"

### Generating counterfactuals

In [56]:
!{py} -u imagenet/generate_data.py --mode random --weights_path {weight_path} --n_data 5 --run_name "random"

Namespace(batch_sz=1, classes=[0, 0, 0], interp='', interp_cls=-1, midpoints=6, mode='random', n_data=5, run_name='random', save_noise=False, save_single=False, truncation=1.0, weights_path='imagenet/weights/cgn.pth')
Saving data to imagenet/data/2022_02_04_16_random_trunc_1.0
100%|█████████████████████████████████████████████| 5/5 [00:30<00:00,  6.11s/it]


In [80]:
from datetime import datetime

data_path = lambda run_name: f'imagenet/data/{datetime.now().strftime("%Y_%m_%d_%H_")}{run_name}_trunc_1.0/ims/'

In [72]:
from IPython.display import Markdown as md
# print()
img_path = data_path("random") + 'random_0000000_x_gen.jpg'

md(f'![counterfactual]({img_path})')

imagenet/data/2022_02_04_16_random_trunc_1.0/ims/random_0000000_x_gen.jpg


![counterfactual](imagenet/data/2022_02_04_16_random_trunc_1.0/ims/random_0000000_x_gen.jpg)

### Generating non-counterfactuals

(Optional) Uncomment the next line if you want to get accurate inception scores and mu mask values. Note that this will increase the run time of the `generate_data` function to at least 4 hours.

In [73]:
n_data = 100
# n_data = 50_000 

In [75]:
!{py} -u imagenet/generate_data.py --mode random_same_class --weights_path {weight_path} --n_data {n_data} --run_name "random_same"

Namespace(batch_sz=1, classes=[0, 0, 0], interp='', interp_cls=-1, midpoints=6, mode='random_same_class', n_data=100, run_name='random_same', save_noise=False, save_single=False, truncation=1.0, weights_path='imagenet/weights/cgn.pth')
Saving data to imagenet/data/2022_02_04_16_random_same_trunc_1.0
100%|█████████████████████████████████████████| 100/100 [08:18<00:00,  4.99s/it]


In [81]:
random_same = data_path("random_same")

The Inception Score is calcualted using TensorFlow because the Pytorch implementation deviates from the results obtained from the original paper.

IS for CGN:

In [84]:
!{py} -u -m imagenet.inception_score --path {random_same} --batch-size 64 --splits 1 --cuda --kind x_gen --tensorflow

2022-02-04 16:37:46.570319: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-04 16:37:46.570373: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Instructions for updating:
non-resource variables are not supported in the long term

2022-02-04 16:37:49.408338: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-04 16:37:49.413155: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open sh

IS for BigGAN:

In [83]:
!{py} -u -m imagenet.inception_score --path {random_same} --batch-size 64 --splits 1 --cuda --kind x_gt --tensorflow

2022-02-04 16:37:23.971756: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-04 16:37:23.971808: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Instructions for updating:
non-resource variables are not supported in the long term

2022-02-04 16:37:26.554255: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-04 16:37:26.555150: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open sh

$\mu_{mask}$ value for the CGN

In [85]:
!{py} -u -m imagenet.calculate_mask --path {random_same}

Loading masks
Computing mean and std pixel values...
Mean: 0.336
Std: 0.185


## Appendix

Table 7

In [ ]:
!{py} experiments/figure_7.py
!{py} experiments/plot_fig7.py